In [43]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re

In [65]:
#conda install -c anaconda pytest


SyntaxError: invalid syntax (<ipython-input-65-16d59d337ad1>, line 2)

In [67]:
#conda install -c conda-forge ipytest

In [71]:
import pytest 
import ipytest
ipytest.autoconfig()

In [146]:
list_fips = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county", match='FIPS', header=0, converters={'FIPS': str})[0]

In [72]:
combined = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/hia_covid_combined.csv", dtype={'County_FIPS': 'str'} )

In [73]:
covid = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/covid_dataset_full.csv", dtype={'County_FIPS': 'str'} )

In [74]:
counties = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/counties_dataset_full.csv", dtype={'County_FIPS': 'str'} )

In [295]:
%%run_pytest[clean]

def county_check(base,next_comp):
    base_fips = set(base.County_FIPS.astype("str").unique())
    comparator_fips = set(next_comp.County_FIPS.astype("str").unique())
    missing = list((base_fips - comparator_fips))
    missing = [str(i) for i in missing] # change to string so that it can be examined 
    r = re.compile("[^72].*") # codes that match puerto rico can't be included
    newlist = list(filter(r.match, missing)) # Read Note
    return newlist

def fips_notna_check(list_dfs):
    list_locs_all = list()
    for i in list_dfs:
        df = i
        df_na = df[df.County_FIPS.isna() == True]
        try:
            list_locs = list(df_na.Name.unique())
            list_locs_all.extend(list_locs)
        except:
            list_locs = list(df_na.locationcol.unique())
            list_locs_all.extend(list_locs)
    return list_locs_all


def test_county_check_1():
    assert county_check(covid, combined) == list()

def test_county_check_2():
    assert county_check(covid, counties) == list()
    
def test_county_check_3():
    assert county_check(counties, covid) == list()
    
def test_county_check_4():
    assert county_check(combined, counties) == list()

def test_fips_notna_check():
    assert fips_notna_check([combined, counties, covid]) == list()


FFF.F                                                                    [100%]
=================================== FAILURES ===================================
_____________________________ test_county_check_1 ______________________________

    def test_county_check_1():
>       assert county_check(covid, combined) == list()
E       AssertionError: assert ['11001', '15...nan', '17099'] == []
E         Left contains 5 more items, first extra item: '11001'
E         Full diff:
E         - []
E         + ['11001', '15005', '51515', 'nan', '17099']

<ipython-input-295-75d45ad5f3ff>:25: AssertionError
_____________________________ test_county_check_2 ______________________________

    def test_county_check_2():
>       assert county_check(covid, counties) == list()
E       AssertionError: assert ['11001', '15...nan', '17099'] == []
E         Left contains 5 more items, first extra item: '11001'
E         Full diff:
E         - []
E         + ['11001', '15005', '51515', 'nan', '17099']

<

In [296]:
fips_notna_check([counties])

[]

In [140]:
missinglist = county_check(counties, covid)

In [142]:
missingfips_df = pd.DataFrame(pd.Series(missinglist), columns = ['FIPS'])
df = missingfips_df.merge(list_fips, on = ["FIPS"])
df

,FIPS,County or equivalent,State or equivalent
0,51067,Franklin County,Virginia
1,02164,Lake and Peninsula Borough[a],Alaska
2,51019,Bedford County[v],Virginia
3,51059,Fairfax County,Virginia
4,51161,Roanoke County,Virginia
5,51159,Richmond County,Virginia


In [186]:
fips_notna_all_test()

AssertionError: assert 1 == 2

In [179]:
fips_notna_check(list_combined)

[]

In [286]:
%%run_pytest[clean]

def negative_check(list_dfs):
    output_dict = dict()
    for df in list_dfs:
        negative_deaths = df[df['Deaths_New'] < 0]
        negative_confirmed = df[df['Confirmed_New'] < 0]
        #df_count = df[df['Deaths'] < 0]
        output_dict['negative_deaths'] = negative_deaths
        output_dict['negative_confirmed']= negative_confirmed
    return output_dict

def test_negative_check():
    output_dict = negative_check([covid])
    assert output_dict['negative_deaths'].shape[0] == 0 and output_dict['negative_confirmed'].shape[0] == 0


F                                                                        [100%]
=================================== FAILURES ===================================
_____________________________ test_negative_check ______________________________

    def test_negative_check():
        output_dict = negative_check([covid])
>       assert output_dict['negative_deaths'].shape[0] == 0 and output_dict['negative_confirmed'].shape[0] == 0
E       assert (2959 == 0)

<ipython-input-286-c994c1ecd00e>:13: AssertionError
=========================== short test summary info ============================
FAILED tmpn5aaywz9.py::test_negative_check - assert (2959 == 0)
1 failed in 0.05s


In [289]:
fr = negative_check([covid])['negative_deaths']

fr.groupby("locationcol")[fr.columns].count().sort_values(by="Date",ascending=False)

,Unnamed: 0,Date,Admin2,State,Confirmed,Deaths,Country/Region,Deaths_Previous,Confirmed_Previous,Deaths_New,...,Confirmed_New_RollingAvg,Deaths_New_RollingAvg,Deaths_New_+7_RollingAvg,Deaths_New_+14_RollingAvg,Deaths_New_+21_RollingAvg,Deaths_New_+28_RollingAvg,County_FIPS,State Code,Region,Division
locationcol,,,,,,,,,,,,,,,,,,,,,
Out of TN|Tennessee,33,33,33,33,33,33,33,33,33,33,...,33,33,32,30,29,29,0,33,33,33
Union|New Jersey,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,18,19,19,19,19
Snohomish|Washington,18,18,18,18,18,18,18,18,18,18,...,18,18,17,16,16,16,18,18,18,18
Bergen|New Jersey,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
Out of GA|Georgia,15,15,15,15,15,15,15,15,15,15,...,15,15,14,14,14,14,0,15,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Monroe|West Virginia,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,1,1,1,1
Montcalm|Michigan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
Monterey|California,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [15]:
# Did we lose any counties

In [16]:
# Did we lose any date ranges entirely?

In [17]:
# Duplicates in data

In [18]:
# Junk data 
## Negative counts of cases 
## Negative deaths 